In [1]:
import pandas as pd
import numpy as np
import os
import sklearn

In [2]:
#assign always returns a copy of the data, leaving the original DataFrame untouched.
#df.columns = [x.lower() for x in df.columns]

In [3]:
df = pd.read_csv("COVID19WW.csv")

# Treat NAN values

In [4]:
df.isnull().sum()

country                         0
country_code                    0
year_week                       0
source                          0
new_cases                      23
number_sequenced                0
percent_cases_sequenced        23
valid_denominator               0
variant                         0
number_detections_variant       0
percent_variant              4602
dtype: int64

### Fill the 'new_cases' with the mean of the previous and following week

In [5]:
#Here, we do not want to fill the NaN values from the 'new_cases' by 0 or mean of the whole column. The column contain
#the value of all countries and weeks. Instead i take the mean of the week before and after.

df_null = df[df['new_cases'].isna()]
df2 = pd.concat([df[['country','year_week','new_cases']].ffill(), df[['country','year_week','new_cases']].bfill()]).groupby(['country','year_week']).mean()
df2.reset_index(inplace = True)

#### Additional stats using the NaN treated

In [6]:
#Compute the cumulative amount of cases per week and per country
df2["newcases_sum"] = df2.groupby(['country'])['new_cases'].apply(lambda x: x.cumsum())

In [13]:
df[(df.country=="Austria") & (df.year_week == "2020-40")]

,country,country_code,year_week,source,new_cases,number_sequenced,percent_cases_sequenced,valid_denominator,variant,number_detections_variant,percent_variant
0,Austria,AT,2020-40,GISAID,5152.0,4,0.1,Yes,B.1.1.7,0,0.0
1,Austria,AT,2020-40,GISAID,5152.0,4,0.1,Yes,B.1.1.7+E484K,0,0.0
2,Austria,AT,2020-40,GISAID,5152.0,4,0.1,Yes,B.1.351,0,0.0
3,Austria,AT,2020-40,GISAID,5152.0,4,0.1,Yes,B.1.427/B.1.429,0,0.0
4,Austria,AT,2020-40,GISAID,5152.0,4,0.1,Yes,B.1.525,0,0.0
5,Austria,AT,2020-40,GISAID,5152.0,4,0.1,Yes,B.1.526,0,0.0
6,Austria,AT,2020-40,GISAID,5152.0,4,0.1,Yes,B.1.616,0,0.0
7,Austria,AT,2020-40,GISAID,5152.0,4,0.1,Yes,B.1.617,0,0.0
8,Austria,AT,2020-40,GISAID,5152.0,4,0.1,Yes,B.1.617.1,0,0.0
9,Austria,AT,2020-40,GISAID,5152.0,4,0.1,Yes,B.1.617.2,0,0.0


In [15]:
df.sort_values(['country', 'year_week'])

,country,country_code,year_week,source,new_cases,number_sequenced,percent_cases_sequenced,valid_denominator,variant,number_detections_variant,percent_variant
0,Austria,AT,2020-40,GISAID,5152.0,4,0.1,Yes,B.1.1.7,0,0.0
1,Austria,AT,2020-40,GISAID,5152.0,4,0.1,Yes,B.1.1.7+E484K,0,0.0
2,Austria,AT,2020-40,GISAID,5152.0,4,0.1,Yes,B.1.351,0,0.0
3,Austria,AT,2020-40,GISAID,5152.0,4,0.1,Yes,B.1.427/B.1.429,0,0.0
4,Austria,AT,2020-40,GISAID,5152.0,4,0.1,Yes,B.1.525,0,0.0
...,...,...,...,...,...,...,...,...,...,...,...
25503,Sweden,SE,2021-28,GISAID,1858.0,0,0.0,Yes,P.1,0,NaN
25504,Sweden,SE,2021-28,GISAID,1858.0,0,0.0,Yes,P.3,0,NaN
25505,Sweden,SE,2021-28,GISAID,1858.0,0,0.0,Yes,Other,0,NaN
25711,Sweden,SE,2021-28,TESSy,1858.0,11,0.6,Yes,B.1.617,0,0.0


In [55]:
#vv = df.set_index(["country","year_week"]).drop_duplicates(subset=['country', 'year_week']keep='first').sort_index()
#xx = df[["country","year_week","number_sequenced"]].groupby(['country','year_week']).mean()
vv = df.drop_duplicates(subset=['country', 'year_week'],keep='first').sort_index()
vv

,country,country_code,year_week,source,new_cases,number_sequenced,percent_cases_sequenced,valid_denominator,variant,number_detections_variant,percent_variant
0,Austria,AT,2020-40,GISAID,5152.0,4,0.1,Yes,B.1.1.7,0,0.0
17,Austria,AT,2020-41,GISAID,7365.0,24,0.3,Yes,B.1.1.7,0,0.0
34,Austria,AT,2020-42,GISAID,9574.0,41,0.4,Yes,B.1.1.7,0,0.0
51,Austria,AT,2020-43,GISAID,16979.0,38,0.2,Yes,B.1.1.7,0,0.0
68,Austria,AT,2020-44,GISAID,28574.0,51,0.2,Yes,B.1.1.7,0,0.0
...,...,...,...,...,...,...,...,...,...,...,...
25421,Sweden,SE,2021-24,GISAID,2840.0,1577,55.5,Yes,B.1.1.7,979,62.1
25438,Sweden,SE,2021-25,GISAID,1980.0,695,35.1,Yes,B.1.1.7,455,65.5
25455,Sweden,SE,2021-26,GISAID,1864.0,164,8.8,Yes,B.1.1.7,69,42.1
25472,Sweden,SE,2021-27,GISAID,1508.0,1,0.1,Yes,B.1.1.7,1,100.0


In [56]:
xxx = pd.merge(left=df2, right=df, left_on=['country'], right_on=['country'])
xxx

,country,year_week_x,new_cases_x,newcases_sum,country_code,year_week_y,source,new_cases_y,number_sequenced,percent_cases_sequenced,valid_denominator,variant,number_detections_variant,percent_variant
0,Austria,2020-40,5152.0,5152.0,AT,2020-40,GISAID,5152.0,4,0.1,Yes,B.1.1.7,0,0.0
1,Austria,2020-40,5152.0,5152.0,AT,2020-40,GISAID,5152.0,4,0.1,Yes,B.1.1.7+E484K,0,0.0
2,Austria,2020-40,5152.0,5152.0,AT,2020-40,GISAID,5152.0,4,0.1,Yes,B.1.351,0,0.0
3,Austria,2020-40,5152.0,5152.0,AT,2020-40,GISAID,5152.0,4,0.1,Yes,B.1.427/B.1.429,0,0.0
4,Austria,2020-40,5152.0,5152.0,AT,2020-40,GISAID,5152.0,4,0.1,Yes,B.1.525,0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1079941,Sweden,2021-28,1858.0,1002085.0,SE,2021-27,TESSy,1508.0,287,19.0,Yes,B.1.617,0,0.0
1079942,Sweden,2021-28,1858.0,1002085.0,SE,2021-27,TESSy,1508.0,287,19.0,Yes,B.1.617.2,226,78.7
1079943,Sweden,2021-28,1858.0,1002085.0,SE,2021-27,TESSy,1508.0,287,19.0,Yes,P.1,1,0.3
1079944,Sweden,2021-28,1858.0,1002085.0,SE,2021-28,TESSy,1858.0,11,0.6,Yes,B.1.617,0,0.0


##### add the number of sequenced and percent cases sequenced
##### for the percent just compute it
##### Then compute the cumulative number sequenced and at the cumulative %

##### Merging the new stats with the 'df' dataframe

In [91]:
merged_left = pd.merge(left=df2, right=df, how='left', left_on=['country','year_week'], right_on=['country','year_week'])
merged_left[(merged_left['country'] == "Spain") & (merged_left['year_week'] == "2021-28")]

,country,year_week,new_cases_x,newcases_sum,country_code,source,new_cases_y,number_sequenced,percent_cases_sequenced,valid_denominator,variant,number_detections_variant,percent_variant
24772,Spain,2021-28,190726.0,3493382.0,ES,GISAID,190726.0,0,0.0,Yes,B.1.1.7,0,NaN
24773,Spain,2021-28,190726.0,3493382.0,ES,GISAID,190726.0,0,0.0,Yes,B.1.1.7+E484K,0,NaN
24774,Spain,2021-28,190726.0,3493382.0,ES,GISAID,190726.0,0,0.0,Yes,B.1.351,0,NaN
24775,Spain,2021-28,190726.0,3493382.0,ES,GISAID,190726.0,0,0.0,Yes,B.1.427/B.1.429,0,NaN
24776,Spain,2021-28,190726.0,3493382.0,ES,GISAID,190726.0,0,0.0,Yes,B.1.525,0,NaN
24777,Spain,2021-28,190726.0,3493382.0,ES,GISAID,190726.0,0,0.0,Yes,B.1.526,0,NaN
24778,Spain,2021-28,190726.0,3493382.0,ES,GISAID,190726.0,0,0.0,Yes,B.1.616,0,NaN
24779,Spain,2021-28,190726.0,3493382.0,ES,GISAID,190726.0,0,0.0,Yes,B.1.617,0,NaN
24780,Spain,2021-28,190726.0,3493382.0,ES,GISAID,190726.0,0,0.0,Yes,B.1.617.1,0,NaN
24781,Spain,2021-28,190726.0,3493382.0,ES,GISAID,190726.0,0,0.0,Yes,B.1.617.2,0,NaN


In [95]:
##### Now need to fill the NaN from number_sequenced and percent_cases_sequenced

In [ ]:
#df.fillna(df.mean().astype(int))
#df.new_cases.mean().astype(int)